<div style="
    border: 2px solid #4CAF50; 
    padding: 15px; 
    background-color: #f4f4f4; 
    border-radius: 10px; 
    align-items: center;">

<h1 style="margin: 0; color: #4CAF50;">Neural Networks: Die Train Methode + Tipps und Tricks</h1>
<h2 style="margin: 5px 0; color: #555;">DSAI</h2>
<h3 style="margin: 5px 0; color: #555;">Jakob Eggl</h3>

<div style="flex-shrink: 0;">
    <img src="https://www.htl-grieskirchen.at/wp/wp-content/uploads/2022/11/logo_bildschirm-1024x503.png" alt="Logo" style="width: 250px; height: auto;"/>
</div>
<p1> © 2025/26 Jakob Eggl. Nutzung oder Verbreitung nur mit ausdrücklicher Genehmigung des Autors.</p1>
</div>
<div style="flex: 1;">
</div>   

In diesem Notebook werden wir uns mit der **Trainingsmethode** befassen. Das bedeutet, dass wir uns ansehen werden, wie normalerweise eine Trainingsmethode aussieht.

Dabei werden wir sowohl:
* Notwendige Schritte besprechen, als auch
* *Tipps und Tricks* besprechen, welche für den reinen Lernprozess nicht notwendig sind, jedoch uns das Leben wesentlich erleichtern können.

## Wiederholung: Welche Punkte müssen wir abarbeiten?

1. Wir definieren unsere Modellklasse und erstellen eine Instanz $f$. Unser Modell hat dabei die Parameter $w$ (genannt *Weights*).
2. Wir haben unsere Daten in einem Dataset und einen zugehörigen Dataloader verfügbar. Dabei steht $X$ für die "Input"-Daten und $y$ das dazugehörige Label.
3. Wir haben eine Loss-Funktion $L(\hat{y},y)$ definiert.
4. Wir haben einen Optimizer (zum Beispiel SGD) und eine Learning Rate $\eta$ festgelegt.
5. Wir schicken die Daten ($X$) durch das Modell und erhalten die Prediction $\hat{y} = f(X)$.
6. Wir vergleichen die echte Lösung $y$ mit unserer Prediction $\hat{y}=f(X)$ indem wir den Loss $L(\hat{y},y)$ berechnen.
7. Wir berechnen die Ableitung der Lossfunktion $L$ bezüglich der Parameter $w$ und updaten diese bezüglich der Regel $w_t = w_{t-1} - \eta \nabla L(w_{t-1})$
8. Wir wiederholen die Schritte 5, 6 und 7 mehrmals.

Versuchen wir nun, dies in Python (PyTorch) zu implementieren.

---

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import torch.optim as optim

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(device)

Nehmen wir nun das `california_housing` Dataset, gespeichert unter `housing.csv`. Dieses wollen wir für Regression verwenden.

In [ ]:
path = os.path.join("..", "..", "_data", "housing.csv")
data = pd.read_csv(path, sep=',', header=0)

(Hier müssten wir jetzt eigentlich noch eine genau Datenanalyse machen. Wir gehen aber jetzt davon aus, dass dies schon gemacht wurde und wir führen somit einfach die folgenden Schritt durch.)

In [ ]:
data

In [ ]:
data.info()

In [ ]:
str_colums = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea","furnishingstatus"]

for col in str_colums:
    print(col,":",data[col].unique())

In [ ]:
bin_colums = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]

for col in bin_colums:
    data[col] = data[col].map({"yes":1,"no":0})

In [ ]:
furnish_encoder = OrdinalEncoder(categories=[["unfurnished","semi-furnished","furnished"]])

data["furnishingstatus"] = furnish_encoder.fit_transform(data[["furnishingstatus"]])

data

In [ ]:
scaler = StandardScaler()

scale_cols = ["area"]

data[scale_cols] = scaler.fit_transform(data[scale_cols])

In [ ]:
data

In [ ]:
data['price'] /= 1e6

In [ ]:
data

In [ ]:
X = data.drop("price", axis=1)
y = data["price"]

In [ ]:
dataset = TensorDataset(torch.tensor(X.values, dtype=torch.float32), torch.tensor(y.values, dtype=torch.float32))

In [ ]:
train_set_ratio = 0.7
test_set_ratio = 0.3

train_length = int(round(train_set_ratio * len(dataset)))
test_length = len(dataset) - train_length
train_dataset, test_dataset = random_split(dataset, [train_length, test_length])

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class SimpleRegressor(nn.Module):
    def __init__(self):
        super(SimpleRegressor, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(12, 24),
            nn.ReLU(),
            nn.Linear(24, 12),
            nn.ReLU(),
            nn.Linear(12, 6),
            nn.ReLU(),
            nn.Linear(6, 1)
            )
    def forward(self, x):
        return self.net(x)

In [ ]:
model = SimpleRegressor().to(device)

Wir werden sehen, dass beim Optimizer die **Model-Parameter** übergeben werden. Dieser Schritt ist sehr wichtig.

Wir sehen uns an dieser Stelle mal die Model-Parameter an.

In [ ]:
# Show number of model parameters and where they come from
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params}")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter: {name}, Shape: {param.shape}, Size: {param.numel()}")

Nun zur eigentlichen **Trainingsmethode**!

In [ ]:
# Hyperparameter

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3) # or SGD for example

n_epochs = 50

**Wichtig:** Hier müssen die Parameter vom Modell beim Optimizer übergeben werden. Diese werden dann im Laufe geupdated.

In [ ]:
print(f"Starting training on device: {device}...")

for epoch in range(1, n_epochs + 1):
    model.train()
    train_losses_this_epoch = []

    for train_input, train_label in train_loader:
        train_input, train_label = train_input.to(device), train_label.to(device)

        optimizer.zero_grad()
        prediction = model(train_input)
        loss = criterion(prediction, train_label)
        loss.backward()
        optimizer.step() 
        train_losses_this_epoch.append(loss.item()) # .item() to get scalar value from tensor.
    avg_train_loss = np.mean(train_losses_this_epoch)
    
    
    model.eval()
    test_losses_this_epoch = []
    with torch.no_grad():
        for test_input, test_label in test_loader:
            test_input, test_label = test_input.to(device), test_label.to(device)

            test_prediction = model(test_input)
            test_loss = criterion(test_prediction, test_label)
            test_losses_this_epoch.append(test_loss.item())
    avg_test_loss = np.mean(test_losses_this_epoch)
    print(f"Epoch {epoch}/{n_epochs} - Train Loss: {avg_train_loss:.4f} - Test Loss: {avg_test_loss:.4f}")

In [ ]:
print(f"Der Testloss beträgt {avg_test_loss:.4f}")

Insbesondere sind folgende Punkte im Code **extrem** wichtig (sortiert der Reihe nach, wie sie in der Methode auftreten):

* `model.train()`: Setzt das Modell in den Trainingsmodus und aktiviert dabei Effekte, welche wir im Training haben wollen und in der späteren Verwendung (=Inferenz) nicht. (Siehe später zum Beispiel **Dropout-Layer**)
* `optimizer.zero_grad()`: Setzt alle Gradienten zurück. Grund: Wir wollen jede Iteration den Gradient neu berechnen und nicht die alten Gradients noch miteinbeziehen.
* `loss = criterion(prediction, train_label)`: Berechnet den Loss, indem es die Prediction und das Label vergleicht.
* `loss.backward()`: Berechnet die Gradienten bezüglich der Parameter (Weights) des Modells (werden vorher in der Initialisierung vom Optimizer übergeben).
* `optimizer.step()`: Aktualisiert die Parameter bzgl. der gewählten **Update-Rule** (zum Beispiel (Stochastic)Gradient-Descent).
* `model.eval()`: Gegenstück zu `model.train()`: Setzt also das Modell in den Evaluierungsmodus, dabei werden einige Effekte, welche wir im Training haben wollen und in der späteren Verwendung nicht, deaktiviert. (Auch hier: Siehe zum Beispiel **Dropout-Layer**)
* `with torch.no_grad()`: Gibt der Autograd-Engine bescheid, dass keine Gradienten gespeichert/berechnet werden sollen. Dies resultiert in einem Geschwindigkeitsboost und einem geringeren Speicherverbrauch. Kann jedoch nur im Evaluierungsmodus verwendet werden. Theoretisch könnten wir somit für das Evaluieren mehr Daten gleichzeitig durch das Modell schicken (größere Batch-Size), nachdem wir weniger Speicher benötigen.

**Wir wollen nochmal kurz die wichtigsten Parameter hier durchgehen:**

* `n_epochs`: Anzahl der Durchläufe des *gesamten* Datasets. Normalerweise schon viel größer als 1. Zu große Anzahl kann zu Overfitting führen. **Lösung:** Early Stopping (siehe später)
* `criterion`: Loss Funktion, welche wir verwenden wollen. Normalerweise Mean Squared Error (MSE) oder Cross Entropy Loss (CE)
* `lr`: Learning Rate, welche vorgibt, wie groß die Schritte in die jeweilige Richtung des steilsten Abstiegs gemacht werden sollen.

**Was ist jetzt noch suboptimal?**

1) Wir verwenden das Testset zwischendurch zum Abfragen der Performance (das ist eigentlich genau genommen auch overfitting).
2) Wir verwenden eine fixe Anzahl an Epochen. Dies kann bei zu kleiner oder zu großer Wahl zu under- bzw. overfitting führen. Außerdem verwenden wir dadurch nicht das Modell, welches zwingend am besten (beim Testset) ist.
3) Wir lassen uns nicht recht viele Parameter zum Trainingsvorgang ausgeben.

Diese Probleme werden wir jetzt Schritt für Schritt lösen.

### Lösung zu Problem 1 und 2 (Mehrfaches Verwenden vom Testset + kein Early Stopping)

> **Übung:** Warum ist das mehrfache Verwenden vom Testset zum Kontrollieren auch overfitting?

Eine mögliche Lösung ist es, ein drittes Dataset einzuführen, das sogenannte **Validierungsset** (**Validationset**).

Es soll verwendet werden, um während des Trainings schon ein Dataset zum Überprüfen der Performance zu haben, mit welchem das Modell **nicht** trainiert worden ist.

Wir verwenden also für neuronale Netze oft 3 verschiedene Datasets:
1) **Trainset:** Mit diesen Daten wird das Netzwerk trainiert.
2) **Validationset:** Mit diesen Daten wird das Netzwerk während dem Trainingsvorgang immer wieder überprüft. Sollte sich die Performance für dieses Dataset nicht mehr ändern, so sollte man den Trainingsvorgang stoppen.
3) **Testset:** Mit diesen Daten wird das Modell final getestet um nun zu sehen, wie gut es "wirklich" ist.

Wir erstellen uns kurz eine Methode, die uns das Dataset generiert:

In [ ]:
def get_dataset(path = os.path.join("..", "..", "_data", "housing.csv")):
    data = pd.read_csv(path, sep=',', header=0)

    bin_colums = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]
    for col in bin_colums:
        data[col] = data[col].map({"yes":1,"no":0})

    furnish_encoder = OrdinalEncoder(categories=[["unfurnished","semi-furnished","furnished"]])

    data["furnishingstatus"] = furnish_encoder.fit_transform(data[["furnishingstatus"]])

    scaler = StandardScaler()

    scale_cols = ["area"]

    data[scale_cols] = scaler.fit_transform(data[scale_cols])

    data['price'] /= 1e6

    X = data.drop("price", axis=1)
    y = data["price"]

    dataset = TensorDataset(torch.tensor(X.values, dtype=torch.float32), torch.tensor(y.values, dtype=torch.float32))

    return dataset


Erstellung eines Validation Sets in PyTorch:

In [ ]:
dataset = get_dataset()

train_set_ratio = 0.7
test_set_ratio = 0.2
validation_set_ratio = 0.1

ratio_sum = train_set_ratio + test_set_ratio + validation_set_ratio

assert np.isclose(ratio_sum, 1.0), f"Ratios must sum to 1.0 but is {ratio_sum}"

train_length = int(round(train_set_ratio * len(dataset)))
test_length = int(round(test_set_ratio * len(dataset)))
validation_length = int(round(validation_set_ratio * len(dataset)))
train_dataset, test_dataset, validation_dataset = random_split(dataset, [train_length, test_length, validation_length])

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

**Hinweis:** Die Verhältnisse 70/20/10 können natürlich auch angepasst werden.

Diese können wir nun nutzen, um ein **Early Stopping** Verhalten zu implementieren.

Mit **Early Stopping** meinen wir folgendes Vorgehen:
* Wir trainieren unser Model auf den Trainings-Daten
* Wir prüfen regelmäßig die aktuelle Performance des Modells am Validation Set
* Sollte sich die Performance auf dem Validation Set über eine gewisse Zeit nicht verbessern, so stoppen wir mit dem Training
* Es wird immer, wenn das Modell besser geworden ist, das aktuelle Modell gespeichert
* Am Schluss wird das beste Modell geladen und wir Testen dieses beste Modell auf dem Testset

In [ ]:
model = SimpleRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Early stopping parameters
patience = 4
best_val_loss = float('inf')
epochs_no_improve = 0
n_epochs = 300

In [ ]:
train_loss_mean_epoch = []
validation_loss_mean_epoch = []

model_export_path = os.path.join("..", "models", "best_model_nn_5.pth")

for epoch in range(1, n_epochs + 1):
    model.train()
    train_losses = []
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
    train_loss_epoch = np.mean(train_losses)
    train_loss_mean_epoch.append(train_loss_epoch) # we save the mean train loss for each epoch to later have a list available if needed for e.g. plotting

    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in validation_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_losses.append(loss.item())
    val_loss_epoch = np.mean(val_losses)
    validation_loss_mean_epoch.append(val_loss_epoch) # we save the mean validation loss for each epoch to later have a list available if needed for e.g. plotting

    print(f"Epoch {epoch}/{n_epochs} — train_loss: {train_loss_epoch:.4f}, val_loss: {val_loss_epoch:.4f}")

    if val_loss_epoch < best_val_loss:
        best_val_loss = val_loss_epoch
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_export_path)
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs.")
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch} epochs. Best val loss: {best_val_loss:.4f}")
            break

**Hinweis:** Wir haben hier schon einen Export vom Model mit `torch.save(model.state_dict(), model_export_path)` verwendet. Wir sehen uns das im nächsten Notebook genauer an!

In [ ]:
# After training, we load the best model and evaluate on the test set
model_path = os.path.join("..", "models", "best_model_nn_5.pth")
model.load_state_dict(torch.load(model_path))

model.eval()
test_losses = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_losses.append(loss.item())
avg_test_loss = np.mean(test_losses)
print(f"Test Loss of the best model: {avg_test_loss:.4f}")

Was genau passiert beim **Early Stopping**?

![Overfitting_Underfitting_Loss_Curve](../resources/Overfitting_Underfitting_Loss_Curve.png)

(von https://www.kaggle.com/code/ryanholbrook/overfitting-and-underfitting)

Wie oben im Bild dargestellt, wollen wir den Punkt erreichen, wo unser Validation-Loss minimal ist. Wir sehen aber, dass nach einer Zeit, der Validation Loss wieder nach **oben geht**, während der Trainingsloss immer weiter nach unten geht.

Der Punkt, an dem der Validation Loss wieder nach oben geht, ist genau der Punkt, bei dem wir zu **overfitten** beginnen. Wir wollen also genau an diesem Punkt mit dem Training aufhören.

Wie können wir das Erreichen?

* Wir berechnen den Validation Loss nach jeder Epoche
* Ist er niedriger, als der bisherige beste Validation Loss, so speichern wir diesen als unseren neuen besten Loss und speichern auch das Modell ab.
* Wird die Validation-Set Performance über eine gewisse Anzahl an Epochen (`patience`) nicht besser, so brechen wir mit dem Training ab.

**Hinweis:** Am Ende zählt trotzdem immer die Testset Performance. Im Normalfall ist aber der Validation Loss ein guter Indikator dafür.

> **Übung:** Ändere die `patience` so, dass wir tatsächlich einen besseren (ähnlichen) Test-Loss erzielen. *Tipp:* Probiere kleine Werte.

**Hinweis:** Um die Problematik, die in der vorigen Übung gezeigt wurde (zu oft "Kontrollieren" mit dem Validationset), zu beheben, wird oft das Validation Set nur alle $n$-Iterationen verwendet zum "Kontrollieren".

### Lösung zu Problem 3 (Zu wenig Parameter des Trainingsvorgangs verfügbar)

Um den Trainingsvorgang besser überwachen zu können, ist es hilfreich, ein Tool wie zum Beispiel [WandB](https://docs.wandb.ai) zu verwenden. Es erlaubt uns, viele weitere Parameter des Trainings zu tracken und im Anschluss in einem Dashboard visuell darzustellen.

Wir sehen uns nun an, wie wir das in unsere Trainingsmethode implementieren können.

Dazu installieren wir die `wandb` (ausgesprochen: Weights & Biases) mit `conda install wandb` bzw. `pip install wandb`.

In [ ]:
import wandb

Zuerst müssen wir uns einloggen.

In [ ]:
wandb.login()

Nun müssen wir ein neues `wandb` Projekt initialisieren 
(siehe zur Anleitung ansonsten auch: [hier](https://docs.wandb.ai/models/tutorials/pytorch)).

**Hinweis:** Pro Initialisierung wird ein Run gespeichert.

In [ ]:
wandb.init(project="Regression_Network", config={
        "learning_rate": 1e-3,
        "batch_size": 32,
        "patience": 10,
        "n_epochs": 300,
        "optimizer": "Adam",
        "criterion": "MSELoss"
    })

In [ ]:
config = wandb.config

In [ ]:
dataset = get_dataset()

train_set_ratio = 0.7
test_set_ratio = 0.2
validation_set_ratio = 0.1

ratio_sum = train_set_ratio + test_set_ratio + validation_set_ratio

assert np.isclose(ratio_sum, 1.0), f"Ratios must sum to 1.0 but is {ratio_sum}"

train_length = int(round(train_set_ratio * len(dataset)))
test_length = int(round(test_set_ratio * len(dataset)))
validation_length = int(round(validation_set_ratio * len(dataset)))
train_dataset, test_dataset, validation_dataset = random_split(dataset, [train_length, test_length, validation_length])

batch_size = config.batch_size

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = SimpleRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

In [ ]:
wandb.watch(model, criterion, log='all', log_freq=10)

In [ ]:
best_val_loss = float('inf')
epochs_no_improve = 0
model_export_path = os.path.join("..", "models", "best_model_nn_5.pth")

for epoch in range(1, config.n_epochs + 1):
    model.train()
    train_losses = []
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
    train_loss_epoch = np.mean(train_losses)
    train_loss_mean_epoch.append(train_loss_epoch)

    ### NEW ###
    metrics = {
        "train/loss": train_loss_epoch,
        "train/epoch": epoch
    }
    wandb.log(metrics)
    ### END NEW ###

    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in validation_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_losses.append(loss.item())
    val_loss_epoch = np.mean(val_losses)

    ### NEW ###
    val_metrics = {
        "val/loss": val_loss_epoch,
        "val/epoch": epoch
    }
    wandb.log(val_metrics)
    ### END NEW ###

    print(f"Epoch {epoch}/{n_epochs} — train_loss: {train_loss_epoch:.4f}, val_loss: {val_loss_epoch:.4f}")

    if val_loss_epoch < best_val_loss:
        best_val_loss = val_loss_epoch
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_export_path)
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs.")
        if epochs_no_improve >= config.patience:
            print(f"Early stopping triggered after {epoch} epochs. Best val loss: {best_val_loss:.4f}")
            break

In [ ]:
# After training, we load the best model and evaluate on the test set
model_path = os.path.join("..", "models", "best_model_nn_5.pth")
model.load_state_dict(torch.load(model_path))

model.eval()
test_losses = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_losses.append(loss.item())
avg_test_loss = np.mean(test_losses)
wandb.summary["test/loss"] = avg_test_loss
print(f"Test Loss of the best model: {avg_test_loss:.4f}")

Nachdem wir mit `wandb` fertig sind, führen wir `wandb.finish()` aus.

In [ ]:
wandb.finish()

Im Anschluss können wir nun auf der Homepage nachsehen, wie sich unsere (Model)Daten während dem Training verhalten haben. 

---

## Tipps und Tricks: Dropout und Batch-Normalization

Um unser Modell noch etwas besser zu machen, lernen wir jetzt noch eine weitere praktische Möglichkeit kennen, welche die Performance unseres Modells verbessern kann. Die Rede ist vom **Dropout**-Layer.

### Dropout

**Idee:**
* Ein neuronales Netzwerk hat **viele Neuronen**, welche gemeinsam ein Muster erlernen.
* Manche Neuronen sind dabei wichtiger als andere, sprich das Modell verlässt sich zu sehr auf einzelne Neuronen.
* Dieses Verhalten neigt zu einer schlechten Generalisierung (i.e. overfitting).
* Um dies zu verbessern, **schalten** wir **zufällig** einen Teil der **Neuronen** während des Trainings **aus**.
* Dadurch muss das Modell lernen, alle Neuronen zu benutzen und nicht nur einen Teil.

![Dropout Visualization](../resources/Dropout_Visualized.png)

(von https://medium.com/@amarbudhiraja/https-medium-com-amarbudhiraja-learning-less-to-learn-better-dropout-in-deep-machine-learning-74334da4bfc5)

Bei der **Evaluierung** wird diese Funktion ausgeschaltet und das Modell hat **alle Neuronen zur Verfügung**.

**Wichtig:** Für andere Architekturen (CNN's, Recurrent Neural Networks (zBsp. LSTM)) muss beim Dropout aufgepasst werden, ob es genau in dieser Form angewendet werden darf.

Wie können wir das in PyTorch verwenden?

* Genauso wie `nn.Linear` gibt es auch `nn.Dropout(p)`
* Dabei muss $p\in[0,1]$ (genannt **Dropout-Rate**) als Parameter übergeben werden.
* Es werden dann $(100\cdot p)$\% der Neuronen zufällig deaktiviert.
* Wir können Dropout jedes Layer verwenden, sollten dabei (normalerweise) aber:
    * es pro Layer immer nach der Aktivierungsfunktion einsetzten.
    * nicht im allerletzten Layer einsetzen.
    * kein zu hohes $p$ verwenden (zu großes $p$ führt zu underfitting)

**Vorteile:**

* Kann gesehen werden, wie wenn wir mehrere verschiedene Modelle trainieren und dann den Durchschnitt nehmen. Dies verringert das Overfitting (Random Forest Idee)
* Einfache Implementierung (wie ein weiteres Layer einfach in die Init-Funktion)

**Nachteile:**

* Pro Trainingsschritt lernt nur ein Teil des Modells, somit sind normalerweise mehr Epochen also auch ein längeres Training notwendig.
* Dropout-Rate ist ein weiterer Hyperparameter, der richtig gewählt werden muss.
* Kann bei falscher Verwendung (insbesondere bei anderen Architekturen) schnell das Modell unbrauchbar machen.

**Wichtig:** Nachdem Dropout nur beim Training angewendet werden soll, ist es extrem wichtig, dass wir beim Modell immer den Modus wechseln mit `model.train()` bzw. `model.eval()`

**Hinweis:** Es gibt auch noch weitere interessante Möglichkeiten, wie **BatchNormalization**, **LayerNormalization**, usw. Diese sehen wir uns nicht genauer an, können aber im Internet recherchiert werden.

Unser angepasstes Modell:

In [ ]:
class SimpleRegressor(nn.Module):
    def __init__(self):
        super(SimpleRegressor, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(12, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
n_epochs = 300
patience = 20
model = SimpleRegressor().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
X = data.drop("price", axis=1)
y = data["price"]

In [ ]:
dataset = TensorDataset(torch.tensor(X.values, dtype=torch.float32), torch.tensor(y.values, dtype=torch.float32))

In [ ]:
train_set_ratio = 0.7
test_set_ratio = 0.2
validation_set_ratio = 0.1

ratio_sum = train_set_ratio + test_set_ratio + validation_set_ratio

assert np.isclose(ratio_sum, 1.0), f"Ratios must sum to 1.0 but is {ratio_sum}"

train_length = int(round(train_set_ratio * len(dataset)))
test_length = int(round(test_set_ratio * len(dataset)))
validation_length = int(round(validation_set_ratio * len(dataset)))
train_dataset, test_dataset, validation_dataset = random_split(dataset, [train_length, test_length, validation_length])

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
best_val_loss = float('inf')
epochs_no_improve = 0

model_export_path = os.path.join("..", "models", "best_model_nn_5_advanced.pth")

train_loss_mean_epoch = []
validation_loss_mean_epoch = []

for epoch in range(1, n_epochs + 1):
    model.train()
    train_losses = []
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    train_loss_epoch = np.mean(train_losses)
    train_loss_mean_epoch.append(train_loss_epoch)

    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_batch, y_batch in validation_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_losses.append(loss.item())
    val_loss_epoch = np.mean(val_losses)
    validation_loss_mean_epoch.append(val_loss_epoch)

    print(f"Epoch {epoch}/{n_epochs} — train_loss: {train_loss_epoch:.4f}, val_loss: {val_loss_epoch:.4f}")

    if val_loss_epoch < best_val_loss:
        best_val_loss = val_loss_epoch
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_export_path)
    else:
        epochs_no_improve += 1
        print(f"No improvement for {epochs_no_improve} epochs.")
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch} epochs. Best val loss: {best_val_loss:.4f}")
            break

In [ ]:
# After training, we load the best model and evaluate on the test set
export_model_path = os.path.join("..", "models", "best_model_nn_5_advanced.pth")
model.load_state_dict(torch.load(export_model_path))

model.eval()
test_losses = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        test_losses.append(loss.item())
avg_test_loss = np.mean(test_losses)
print(f"Test Loss of the best model: {avg_test_loss:.4f}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_loss_mean_epoch, label='Training Loss')
plt.plot(validation_loss_mean_epoch, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.show()

### Skip-Connections

In vielen Netzwerken kann es vom Vorteil sein, sogenannte Skip-Connections zu verwenden. Sie sind ein Shortcut für das Netzwerk, sprich die Daten laufen sowohl durch das Netzwerk, als auch am Netzwerk vorbei und werden später wieder kombiniert. Dies hat sich in vielen Fällen als Vorteilhaft erwiesen und wird oft verwendet. Folgendes Bild zeigt eine beispielhafte Verwendung einer Skip-Connection über 2 Layers.

![Skip_Connections](../resources/Skip_Connections.png)

(von https://theaisummer.com/skip-connections/)

**Mathematische Formulierung einer Skip-Connection:**

Als Formel haben wir
$$y = F(x)+x,$$

dabei ist $x$ der Input des Layers $F$ repräsentiert die Funktion des Layers. $y$ ist der resultierende Output, welcher also eine Summe aus dem ursprünglichen Input und dem Output des Layers ist.

**Vorteile:**

* Netzwerk muss nur den Unterschied lernen.
* Löst zu einem gewissen Grad das **Vanishing-Gradient** Problem, weil dann auch der Gradient diese Abkürzung hat.

**Nachteil(e):**

* Die Dimensionen müssen zusammen passen, ansonsten können die beiden Outputs nicht zusammen addiert werden

> **Übung:** Überlege dir Beispiele, wo Skip-Connections extrem Sinn machen können. Insbesondere, wenn du die eben genannten Nachteile betrachtest.

**Hinweis:** Es gibt auch Ansätze, wo dann an den Verbindungsstellen (oben im Bild dargestellt als $\oplus$) die Ergebnisse nicht addiert, sondern einfach Verkettet werden.

**Beispielhafte Verwendung einer Skip-Connection in PyTorch:**

In [ ]:
class SimpleRegressorwithSkipConnection(nn.Module):
    def __init__(self):
        super(SimpleRegressorwithSkipConnection, self).__init__()
        self.layer1 = nn.Linear(12, 24)
        self.layer2 = nn.Linear(24, 12)
        self.layer3 = nn.Linear(12, 6)
        self.layer4 = nn.Linear(6, 1)

    def forward(self, x):
        layer1_out = F.relu(self.layer1(x))
        layer2_out = F.relu(self.layer2(layer1_out) + x)
        layer3_out = F.relu(self.layer3(layer2_out))
        output = self.layer4(layer3_out)
        return output

**Hinweis:** Es können mehrere "Skip-Verbindungen" eingebaut werden, diese müssen dann einfach in der `forward()` Methode im Netzwerk richtig eingesetzt werden.

### Tracken von Metriken im Trainingsprozess

Eine weitere praktische Sache ist, sich auch Metriken im Training anzeigen zu lassen. So können wir zum Beispiel nicht nur den Loss tracken, sondern auch zum Beispiel die **Accuracy**.

Dabei müssen wir nur an jenen Stellen, wo der Loss berechnet wird, zusätzlich auch noch die Metrik berechnen und ausgeben. 

**Hinweis:** Es kann auch ein Early Stopping bzgl. einer Metrik implementiert werden.

---

## Zusammenfassung Hyperparameter

* Batch Size
* Modell Architektur (Layers, MLP, Aktivierungsfunktionen, Dropout usw.)
* Learning Rate (Learning Rate Scheduler)
* Optimizer (SGD, Adam, Adagrad)
* Transformation von Daten
* Early Stopping
* Train/Test Split
* Loss-Funktion

![Reel_First_Try_Suspicious](../resources/Instagram_Reel_First_Try_Suspicious.mp4)

(von https://www.instagram.com/reel/DNiWMPWSjHc/?igsh=MW04OXNlczZjcHVydQ==)